In [1]:
USE_SAME_FEW_SHOTS=True

# INSTALLAZIONE E IMPORT LIBRERIE

In [1]:
import pandas as pd
import glob
import pickle

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
# input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx"
# input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"


output_data_path = base_path + "/../Llama-2/data/output/2023.11.21-4_LLama2_13B-chat-hf_4-bit_float-16_3-few-shot"

drive.mount(mount_point, force_remount=True)
# data_path = '/content/gdrive/MyDrive/Colab/SDG/Data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx'

Mounted at /content/gdrive


In [3]:
# model_id = "meta-llama/Llama-2-7b-chat-hf"
model_id = "meta-llama/Llama-2-13b-chat-hf"

In [4]:
nbit = 4
# llamasize = '7B'
llamasize = '13B'
temp = 0.1
# outdir = 'data/output/2023.10.20-2_LLama2_langchain/'
# outdir = 'data/output/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/'
# outfilename = outdir+f'sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}.xlsx'
# outfilename

In [7]:
#!pip install --upgrade pip
!pip install transformers # Download e caricamento modelli
!pip install bitsandbytes # Quantizzazione
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00


In [8]:
# Installazione "permanente"
!pip install sentencepiece && touch ~/.ipython/profile_default/startup/00-first.py && echo "import <sentencepiece>" >> ~/.ipython/profile_default/startup/00-first.py

In [9]:
!pip install pillow==10.0.1
!pip install langchain[llms]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.0/631.0 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [10]:
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [11]:
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [12]:
torch.cuda.empty_cache()

# LOGIN DA HUGGING FACE

In [16]:
from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [17]:
!huggingface-cli login --token $huggingface_cli_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# DOWNLOAD E CARICAMENTO DEL MODELLO

In [18]:
# pip install --upgrade bitsandbytes

In [19]:
model = LlamaForCausalLM.from_pretrained(  # Caricamento del modello usando la libreria dedicata. In questo caso, "ForCausal" si riferisce al modello per la generazione di testo
    model_id, # Nome del modello precedentemente impostato
    load_in_4bit = True, # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
    torch_dtype = torch.float16 # Impostare questa codifica aiuta il modello in fase di generazione, in quanto il prompt è tipicamente float16
)

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [20]:
torch.cuda.empty_cache()

In [21]:
!nvidia-smi

Fri Feb 16 17:27:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0              30W /  70W |   7785MiB / 15360MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [22]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [23]:
torch.cuda.empty_cache()

In [24]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [25]:
torch.cuda.empty_cache()

## TEST

In [26]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [27]:
torch.cuda.empty_cache()

In [28]:
llm = HuggingFacePipeline(pipeline=generate_text)

In [29]:
torch.cuda.empty_cache()

In [5]:
# data = pd.read_excel("data/input/data_sel_4_few_shot_examples.xlsx")
# data = pd.read_excel("data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx")
# data = pd.read_excel("../data/input/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = pd.read_excel(input_data_path)
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125
3,3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146
4,4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124
...,...,...,...,...,...,...,...,...,...,...
1015,1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100
1016,1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60
1017,1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124
1018,1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109


In [31]:
data.shape

(1020, 10)

In [32]:
data.groupby('sdg').count()['text']

sdg
0     60
1     60
2     60
3     60
4     60
5     60
6     60
7     60
8     60
9     60
10    60
11    60
12    60
13    60
14    60
15    60
16    60
Name: text, dtype: int64

In [6]:
data_sel = data

In [34]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) #|
        # (data_sel_4_few_shot_examples.text_id==370)  |
        # (data_sel_4_few_shot_examples.text_id==1422)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(5)

data_sel_4_few_shot_examples

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
2319,2319,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.0000,53
3569,3569,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.8125,47
4608,4608,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.0000,136


In [35]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

print(few_shots)

Input text: "As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far."
Output: "---SDG 8---"

Input text: "This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
Output: "---SDG 12---"

Input text: "The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was a

In [36]:
template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are 3 examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
{text}
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""

In [37]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [38]:
torch.cuda.empty_cache()

In [39]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 0
results = []
for index, row in data_sel.iterrows():

    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]

    try:
      result = llm(prompt.format(text=text))
    except Exception as e:
      print(cnt, text_id, sdg_true, e)

    result_extended = (cnt, text_id, sdg_true, result)

    print(result_extended)

    with open(output_data_path+f'/sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)

    results.append(result_extended)

    cnt+=1
    #time.sleep(0.5)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


(0, 6668, 0, '---SDG 11---')
(1, 6099, 0, '---SDG 12---')
(2, 6471, 0, '---SDG 16---')
(3, 1786, 0, '---SDG 4---')
(4, 2143, 0, '---SDG 8---')
(5, 3377, 0, '---SDG 10---')
(6, 5589, 0, '---SDG 9---')
(7, 5431, 0, '---SDG 8---')
(8, 768, 0, '---SDG 12---')
(9, 2735, 0, '---SDG 11---')
(10, 1306, 0, '---SDG 16---')
(11, 3844, 0, '---SDG 12---')
(12, 1524, 0, '---SDG 11---')
(13, 4660, 0, '---SDG 3---')
(14, 5077, 0, '---SDG 16---')
(15, 995, 0, '---SDG 11---')
(16, 2166, 0, '---SDG 16---')
(17, 2512, 0, '---SDG 16---')
(18, 2191, 0, '---SDG 11---')
(19, 6073, 0, '---SDG 13---')
(20, 1376, 0, '---SDG 13---')
(21, 3889, 0, '---SDG 12---')
(22, 6576, 0, '---SDG 3---')
(23, 3912, 0, '---SDG 8---')
(24, 1565, 0, '---SDG 12---')
(25, 1033, 0, '---SDG 16---')
(26, 5653, 0, '---SDG 8---')
(27, 940, 0, '---SDG 16---')
(28, 3613, 0, "---SDG 12---\n\nThe text you provided does not directly relate to any specific Sustainable Development Goal (SDG), but it does touch on several themes that are releva

In [7]:
files = glob.glob(output_data_path + "/*.pkl")

In [8]:
results_complete = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results_complete.append(res)
    # break

In [9]:
results_complete[:5]

[(20, 1376, 0, '---SDG 13---'),
 (21, 3889, 0, '---SDG 12---'),
 (22, 6576, 0, '---SDG 3---'),
 (23, 3912, 0, '---SDG 8---'),
 (24, 1565, 0, '---SDG 12---')]

In [10]:
results = results_complete

In [11]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed

,cnt,text_id,sdg_true,result
0,20,1376,0,---SDG 13---
1,21,3889,0,---SDG 12---
2,22,6576,0,---SDG 3---
3,23,3912,0,---SDG 8---
4,24,1565,0,---SDG 12---
...,...,...,...,...
1015,15,995,0,---SDG 11---
1016,16,2166,0,---SDG 16---
1017,17,2512,0,---SDG 16---
1018,18,2191,0,---SDG 11---


In [12]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed

,cnt,text_id,sdg_true,result,label
0,20,1376,0,---SDG 13---,13
1,21,3889,0,---SDG 12---,12
2,22,6576,0,---SDG 3---,3
3,23,3912,0,---SDG 8---,8
4,24,1565,0,---SDG 12---,12
...,...,...,...,...,...
1015,15,995,0,---SDG 11---,11
1016,16,2166,0,---SDG 16---,16
1017,17,2512,0,---SDG 16---,16
1018,18,2191,0,---SDG 11---,11


In [13]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,sdg_true,result,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,0,---SDG 11---,11
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,0,---SDG 12---,12
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,0,---SDG 16---,16
3,3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,0,---SDG 4---,4
4,4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,0,---SDG 8---,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,16,---SDG 16---,16
1016,1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,16,---SDG 17---,17
1017,1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,16,---SDG 16---,16
1018,1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,16,---SDG 16---,16


In [14]:
# final.to_excel(outfilename)
final.to_excel(output_data_path+"/2023.11.21-4_LLama2_13B-chat-hf_4-bit_float-16_3-few-shot.xlsx")

In [15]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.75      0.63      0.68        60
           2       0.95      0.65      0.77        60
           3       0.72      0.93      0.81        60
           4       0.76      0.93      0.84        60
           5       0.90      0.78      0.84        60
           6       0.98      0.70      0.82        60
           7       0.91      0.67      0.77        60
           8       0.43      0.73      0.54        60
           9       0.68      0.65      0.67        60
          10       0.58      0.47      0.52        60
          11       0.68      0.90      0.78        60
          12       0.59      0.77      0.67        60
          13       0.67      0.80      0.73        60
          14       0.97      0.55      0.70        60
          15       0.68      0.88      0.77        60
          16       0.69      0.83      0.76        60
          17     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
# classification_report_filename = output_data_path+f'/sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [16]:
# test = pd.read_excel(classification_report_filename)
# test

In [17]:
final_new = final

In [18]:
results_processed_merged = final_new
results_processed_merged = results_processed_merged.rename(columns={'label': 'predicted_label_clean'})

In [19]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

# NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# df.loc[df['label'].isna(), 'label'] = df.loc[df['label'].isna(), 'sdg'] + 1

# NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

In [20]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.75      0.63      0.68        60
           2       0.95      0.65      0.77        60
           3       0.72      0.93      0.81        60
           4       0.76      0.93      0.84        60
           5       0.90      0.78      0.84        60
           6       0.98      0.70      0.82        60
           7       0.91      0.67      0.77        60
           8       0.43      0.73      0.54        60
           9       0.68      0.65      0.67        60
          10       0.58      0.47      0.52        60
          11       0.68      0.90      0.78        60
          12       0.59      0.77      0.67        60
          13       0.67      0.80      0.73        60
          14       0.97      0.55      0.70        60
          15       0.68      0.88      0.77        60
          16       0.69      0.83      0.76        60

    accuracy    

In [21]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100

In [22]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [23]:
print(precision, recall, accuracy, f1_score)

70.23474717765802 69.90196078431373 69.90196078431372 68.54251253594633


In [24]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

70.2 69.9 69.9 68.5
